In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

from glob import glob
import json
import csv
import os

import cv2
import numpy as np

In [31]:
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Convolution2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import LeakyReLU, ELU, Dropout

In [4]:
from keras.optimizers import Adam
from keras.layers import Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
from utils import read_samples
from utils import ImageGenerator
from utils import CenterImageGenerator

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
ROOT_DIR = os.path.join(os.getcwd(), '../')
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [15]:
samples = read_samples(DATA_DIR)
samples.pop(0)

Loaded 8037 samples.


['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

In [16]:
def append_path(line):
    line[0] = os.path.join(DATA_DIR, 'sample-data', line[0].strip())
    line[1] = os.path.join(DATA_DIR, 'sample-data', line[1].strip())
    line[2] = os.path.join(DATA_DIR, 'sample-data', line[2].strip())
    return line

In [17]:
samples = [append_path(line) for line in samples]

In [18]:
samples[0]

['/home/carnd/CarND-Behavioral-Cloning-P3/notebooks/../data/sample-data/IMG/center_2016_12_01_13_30_48_287.jpg',
 '/home/carnd/CarND-Behavioral-Cloning-P3/notebooks/../data/sample-data/IMG/left_2016_12_01_13_30_48_287.jpg',
 '/home/carnd/CarND-Behavioral-Cloning-P3/notebooks/../data/sample-data/IMG/right_2016_12_01_13_30_48_287.jpg',
 ' 0',
 ' 0',
 ' 0',
 ' 22.14829']

In [19]:
train_set, test_set = train_test_split(samples, test_size=0.2)
train_set, valid_set = train_test_split(train_set, test_size=0.2)

In [80]:
train_iterator = ImageGenerator(train_set, batch_size=64, 
                                      flip=True, gray=False, left_corr=0.1, right_corr=0.1)
valid_iterator = ImageGenerator(valid_set, batch_size=64, 
                                      flip=True, gray=False, left_corr=0.1, right_corr=0.1)
test_iterator = CenterImageGenerator(test_set, batch_size=64, 
                                     gray=False)

In [81]:
def comma_model():
    ch, row, col = 1, 160, 320  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
    model.add(ELU())
    model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
    model.add(ELU())
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")
    return model

In [82]:
model = comma_model()

In [83]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_12 (Lambda)           (None, 160, 320, 1)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 40, 80, 16)        1040      
_________________________________________________________________
elu_37 (ELU)                 (None, 40, 80, 16)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 20, 40, 32)        12832     
_________________________________________________________________
elu_38 (ELU)                 (None, 20, 40, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 10, 20, 64)        51264     
_________________________________________________________________
flatten_10 (Flatten)         (None, 12800)             0         
__________

In [84]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, 
                               patience=2, verbose=1, mode='auto')
filepath = 'comma.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, 
                save_weights_only=False, mode='auto', period=1)

In [85]:
model.fit_generator(generator=train_iterator, epochs=10, 
                    validation_data=valid_iterator,
                    steps_per_epoch=len(train_iterator), 
                    validation_steps=len(valid_iterator),
                    callbacks=[early_stopping, checkpoint]
                   )

Epoch 1/10
483/483 [==============================] - 69s - loss: 0.6907 - val_loss: 0.0399
Epoch 2/10
483/483 [==============================] - 68s - loss: 0.0128 - val_loss: 0.0226
Epoch 3/10
483/483 [==============================] - 68s - loss: 0.0097 - val_loss: 0.0232
Epoch 4/10
483/483 [==============================] - 68s - loss: 0.0083 - val_loss: 0.0293
Epoch 5/10
483/483 [==============================] - 69s - loss: 0.0068 - val_loss: 0.0139
Epoch 6/10
483/483 [==============================] - 68s - loss: 0.0059 - val_loss: 0.0240
Epoch 7/10
483/483 [==============================] - 68s - loss: 0.0049 - val_loss: 0.0170
Epoch 8/10
483/483 [==============================] - 69s - loss: 0.0048 - val_loss: 0.0179
Epoch 00007: early stopping


In [86]:
results = model.evaluate_generator(generator=test_iterator, steps=len(test_iterator))

In [87]:
results

0.020940254012552593